# DattoRMM - API - Monthly Reports

# DataFrame Creation from API
## Prepare for DataFrame Creation
### Set Export Folder

In [ ]:
import datetime as dt
import yaml
from yaml import SafeLoader

In [ ]:
# add current timestamp to filename for reference
current_time = (dt.datetime.utcnow().strftime('%Y_%m_%d_%H%M%S'))

# export folder will contain all csv exported DataFrames for Ticket Creation
export_folder = 'd:/exports'

# git location
git_dir = 'd:/git'

# dictionary location
dict_dir = f'{git_dir}/data_parsing/dictionaries'

In [ ]:
from ts_api_connections import datto_rmm_api

## Create DataFrame via API Call Iteration


In [ ]:
datto = datto_rmm_api.DattoRMM()
df_devices = datto.create_devices_dataframe()

# Create reports and save to CSV

## Pull year month to add to csv file title

In [ ]:
# add current timestamp to filename for reference
current_time = (dt.datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S'))
year_month = (dt.datetime.utcnow().strftime('%Y - %m'))
year_month

## Import Client Service Files

In [ ]:
def load_yaml(source_file=f'{dict_dir}.yaml'):
    with open(source_file, "r") as y:
        yaml_contents = yaml.load(y, Loader=SafeLoader)

        return yaml_contents

report_configs = load_yaml()

## Create report dataframes based on site and column filtering and export

In [ ]:
def columns(report,yaml_configs=report_configs):

    try:
        yaml_columns = (yaml_configs['reports'][report]['standard_columns']) + (yaml_configs['reports'][report]['additional_columns'])
    except:
        yaml_columns = yaml_configs['reports'][report]['standard_columns']

    return yaml_columns

In [ ]:
def site_names(report,yaml_configs=report_configs):

    yaml_sites = []

    for yaml_site in yaml_configs['site_names']:

        if report in yaml_site['reports']:

            yaml_sites.append(yaml_site['name'])

    return yaml_sites

In [ ]:
def create_reports(report_list,source_df=df_devices,yaml_configs=report_configs):
    for report in report_list:

        df = source_df[columns(report)]
        df['reportCreationDate'] = current_time

        for yaml_site in site_names(report):

            df[df['siteName'] == yaml_site].to_csv(f'{export_folder}/reports/{yaml_site} - {year_month} - {yaml_configs["reports"][report]["report_name"]}.csv',index=False)

In [ ]:
reports_list = ['ms_patching','software_management','sophos_av']

create_reports(reports_list)